<div style="text-align: right"> <i> Бронников Егор ПМ-1901 </i> </div>
<div style="text-align: right"> <i> Рослая Ирина ПМ-1901 </i> </div>

<h1 align="center"> <img style="float: left"src="./logo.svg" width=60 height=60/> Оптимизация Python кода </h1>

## 0)  Предварительная работа

### 0. Зависимости

In [1]:
import copy
import warnings
import numpy as np
import pandas as pd
from typing import Union
from random import random
from functools import wraps
from time import time

### 1. Создание данных

In [2]:
generate_data_A = lambda n: [[random() for _ in range(n)] for _ in range(n)]

In [3]:
generate_data_f = lambda n: [random() for _ in range(n)]

### 2. Любимые декораторы 😍 💖

In [4]:
def timeit_deco(iters = 1000000):
    def timeit_inner(function):
        @wraps(function)
        def inner(*args, **kwargs):
            start = time()
            result = function(*args, **kwargs)
            for _ in range(iters-1):
                function(*args, **kwargs)
            end = time()
            print(f"Time of {function.__name__}: {end-start} sec, {iters} loops")
            return result
        return inner
    return timeit_inner

## 1) Что мы будем оптимизировать

### 1. Метод Гаусса с частичным выбором ведущего элемента

**Алгоритмы решения СЛАУ:**
1. Метод Гаусса с частичным выбором ведущего элемента
1. Метод наискорейшего спуска

## 2) Метод Гаусса с частичным выбором ведущего элемента

### 0) Предварительная работа

*Входные данные:*

In [5]:
A = [[1.00, 0.17, -0.25, 0.54],
     [0.47, 1.00, 0.67, -0.32],
     [-0.11, 0.35, 1.00, -0.74],
     [0.55, 0.43, 0.36, 1.00]]

In [6]:
f = [0.3, 0.5, 0.7, 0.9]

In [7]:
rand_A = generate_data_A(100)
np_rand_A = np.matrix(rand_A, dtype=np.dtype(np.float64))

In [8]:
rand_f = generate_data_f(100)
np_rand_f = np.array(rand_f, dtype=np.dtype(np.float64))

### 1) Чистный Python

In [9]:
def gaussian_elimination_clear(A_arg: list, f_arg: list) -> list:
    A, f = copy.deepcopy(A_arg), copy.deepcopy(f_arg)
    for i in range(len(A)):
        column = list(map(abs, [row[i] for row in A][i:]))
        if max(column) == 0.:
            warnings.warn("Determinant equals 0")
            return
        max_row = max(range(len(column)), key=column.__getitem__)
        if max_row != 0:
            pos_max = max_row + i
            A[i], A[pos_max] = A[pos_max], A[i]
            f[i], f[pos_max] = f[pos_max], f[i]
        for j in range(i+1, len(A)):
            coef = -(A[j][i]/A[i][i])
            A[j] = [coef * A[i][k] + A[j][k] for k in range(len(A[i]))]
            f[j] = coef * f[i] + f[j]
    n = len(f) 
    x = [0 for _ in range(len(f))]
    x[n-1] = f[n-1]/A[n-1][n-1]
    for i in range(n-2, -1, -1):
        sum_elem = sum(A[i][j] * x[j] for j in range(i+1, n))
        x[i] = (f[i] - sum_elem)/A[i][i]
    return x

In [10]:
gaussian_elimination_clear_verified = timeit_deco()(gaussian_elimination_clear)
gaussian_elimination_clear_verified(A, f)

Time of gaussian_elimination_clear: 26.439902305603027 sec, 1000000 loops


[0.4408885508918321,
 -0.36303099013644724,
 1.166798332275979,
 0.3935672231488123]

In [11]:
gaussian_elimination_clear_big_data = timeit_deco(1000)(gaussian_elimination_clear)
gaussian_elimination_clear_big_data(rand_A, rand_f)

KeyboardInterrupt: 

### 2) Щепото4ка numpy

*Входные данные:*

In [ ]:
np_A = np.matrix(A, dtype=np.dtype(np.float64))

In [ ]:
np_f = np.array(f, dtype=np.dtype(np.float64))

In [ ]:
def gaussian_elimination_numpy(A_arg: np.matrix, f_arg: Union[np.matrix, np.array]) -> Union[np.matrix, np.array]:
    A, f = np.copy(A_arg), np.copy(f_arg)
    for i in range(len(A)):
        column = np.abs(A[i:, i])
        leading_elem = np.max(column)
        if leading_elem == 0.:
            warnings.warn("Determinant equals 0")
            return
        if np.where(column == leading_elem)[0][0] != 0:
            pos_max = column.argmax() + i
            A[[i, pos_max]] = A[[pos_max, i]]
            f[[i, pos_max]] = f[[pos_max, i]]
        for j in range(i+1, len(A)):
            coef = -(A[j, i]/A[i, i])
            A[j] = coef * A[i] + A[j]
            f[j] = coef * f[i] + f[j]
    n = f.shape[0]
    X = np.zeros(shape=f.shape)
    X[n-1] = f[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum_elem = sum(A[i, j] * X[j] for j in range(i+1, n))
        X[i] = (f[i] - sum_elem)/A[i, i]
    return X

In [ ]:
gaussian_elimination_numpy_verified = timeit_deco()(gaussian_elimination_numpy)
gaussian_elimination_numpy_verified(np_A, np_f)

In [ ]:
gaussian_elimination_numpy_big_data = timeit_deco(1000)(gaussian_elimination_numpy)
gaussian_elimination_numpy_big_data(np_rand_A, np_rand_f)

### 3) Чистый Python + Cython

In [ ]:
%load_ext cython

In [ ]:
%%cython -a
import copy
import warnings


def gaussian_elimination_clear_cython(A_arg: list, f_arg: list) -> list:
    A, f = copy.deepcopy(A_arg), copy.deepcopy(f_arg)
    for i in range(len(A)):
        column = [row[i] for row in A][i:]
        if max(column) == 0.:
            warnings.warn("Determinant equals 0")
            return
        max_row = max(range(len(column)), key=column.__getitem__)
        if max_row != 0:
            pos_max = max_row + i
            A[i], A[pos_max] = A[pos_max], A[i]
            f[i], f[pos_max] = f[pos_max], f[i]
        for j in range(i+1, len(A)):
            coef = -(A[j][i]/A[i][i])
            A[j] = [coef * A[i][k] + A[j][k] for k in range(len(A[i]))]
            f[j] = coef * f[i] + f[j]
    n = len(f) 
    x = [0 for _ in range(len(f))]
    x[n-1] = f[n-1]/A[n-1][n-1]
    for i in range(n-2, -1, -1):
        sum_elem = sum(A[i][j] * x[j] for j in range(i+1, n))
        x[i] = (f[i] - sum_elem)/A[i][i]
    return x

In [ ]:
gaussian_elimination_clear_cython_verified = timeit_deco()(gaussian_elimination_clear_cython)
gaussian_elimination_clear_cython_verified(A, f)

In [ ]:
gaussian_elimination_clear_cython_big_data = timeit_deco(1000)(gaussian_elimination_clear_cython)
gaussian_elimination_clear_cython_big_data(rand_A, rand_f)

### 4) Щепото4ка numpy + Cython

In [ ]:
%%cython -a
import warnings
import numpy as np
from typing import Union


def gaussian_elimination_numpy_cython(A_arg: np.matrix, f_arg: Union[np.matrix, np.array]) -> Union[np.matrix, np.array]:
    A, f = np.copy(A_arg), np.copy(f_arg)
    for i in range(len(A)):
        column = np.abs(A[i:, i])
        leading_elem = np.max(column)
        if leading_elem == 0.:
            warnings.warn("Determinant equals 0")
            return
        if np.where(column == leading_elem)[0][0] != 0:
            pos_max = column.argmax() + i
            A[[i, pos_max]] = A[[pos_max, i]]
            f[[i, pos_max]] = f[[pos_max, i]]
        for j in range(i+1, len(A)):
            coef = -(A[j, i]/A[i, i])
            A[j] = coef * A[i] + A[j]
            f[j] = coef * f[i] + f[j]
    n = f.shape[0]
    X = np.zeros(shape=f.shape)
    X[n-1] = f[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum_elem = sum(A[i, j] * X[j] for j in range(i+1, n))
        X[i] = (f[i] - sum_elem)/A[i, i]
    return X

In [ ]:
gaussian_elimination_numpy_cython_verified = timeit_deco()(gaussian_elimination_numpy_cython)
gaussian_elimination_numpy_cython_verified(np_A, np_f)

In [ ]:
gaussian_elimination_numpy_cython_big_data = timeit_deco(1000)(gaussian_elimination_numpy_cython)
gaussian_elimination_numpy_cython_big_data(np_rand_A, np_rand_f)

### 5) Итоги

In [ ]:
pd.read_json("gaussian_verified_data.json").sort_values(by="time (sec)").style.set_properties(**{'font-size': '16pt'})

In [ ]:
pd.read_json("gaussian_big_data.json").sort_values(by="time (sec)").style.set_properties(**{'font-size': '16pt'})

## 3) Метод наискорейшего спуска

### 0. Предварительная работа

*Входные данные:*

In [ ]:
A = [[4.33, -1.12, -1.08, 1.14],
     [-1.12, 4.33, 0.24, -1.22],
     [-1.08, 0.24, 7.21, -3.22],
     [1.14, -1.22, -3.22, 5.43]]

np_A = np.matrix(A, dtype=np.dtype(np.float64))

In [ ]:
f = [0.3, 0.5, 0.7, 0.9]

np_f = np.array(f, dtype=np.dtype(np.float64))

*Умножение матриц*

In [ ]:
def matmul(mat1, mat2):
    return [[sum(mat1[i][k] * mat2[k][j] for k in range(len(mat1[0])))
             for j in range(len(mat2[0]))]
             for i in range(len(mat1))]

*Скалярное произведение*

In [ ]:
def dot(mat1, mat2):
    return sum(x[0] * y[0] for x, y in zip(mat1, mat2))

### 1) Чистый Python

In [ ]:
def steepest_descent_method_clear(A_arg: list, f_arg: list, K_max: int) -> list:
    A, f = copy.deepcopy(A_arg), copy.deepcopy(f_arg)
    x = [[0] for _ in range(len(f))]
    for k in range(K_max):
        mul = matmul(A, x)
        r = [[f[i] - mul[i][0]] for i in range(len(f))]
        alpha = dot(r, r)/dot(matmul(A, r), r)
        x = [[x[i][0] + alpha * r[i][0]] for i in range(len(x))]
    return x

In [ ]:
steepest_descent_method_clear_verified = timeit_deco()(steepest_descent_method_clear)
steepest_descent_method_clear_verified(A, f, 10)

### 2) numpy

In [ ]:
def steepest_descent_method_numpy(A_arg: np.matrix, f_arg: np.array, K_max: int) -> np.array:
    A, f = np.copy(A_arg), np.copy(f_arg)
    if not np.all(np.linalg.eigvals(A) > 0):
        warnings.warn("Matrix is not positive definite")
        return
    elif not np.allclose(A, A.T):
        warnings.warn("Matrix is not symmetric")
        return
    elif K_max < 0:
        warnings.warn("The number of iterations cannot be negative")
        return
    x = np.zeros(f.shape, dtype=np.dtype(np.float64))
    for k in range(K_max):
        r = np.squeeze(np.asarray(f - np.matmul(A, x)))
        alpha = (np.dot(r, r)/np.dot(np.matmul(A, r), r)).item(0)
        x = x + alpha * r
    return x

In [ ]:
steepest_descent_method_numpy_verified = timeit_deco()(steepest_descent_method_numpy)
steepest_descent_method_numpy_verified(np_A, np_f, 10)

### 3) Чистый Python + Cython

In [ ]:
%%cython -a
import copy
from __main__ import matmul
from __main__ import dot


def steepest_descent_method_clear_cython(A_arg: list, f_arg: list, K_max: int) -> list:
    A, f = copy.deepcopy(A_arg), copy.deepcopy(f_arg)
    x = [[0] for _ in range(len(f))]
    for k in range(K_max):
        mul = matmul(A, x)
        r = [[f[i] - mul[i][0]] for i in range(len(f))]
        alpha = dot(r, r)/dot(matmul(A, r), r)
        x = [[x[i][0] + alpha * r[i][0]] for i in range(len(x))]
    return x

In [ ]:
steepest_descent_method_clear_cython_verified = timeit_deco()(steepest_descent_method_clear_cython)
steepest_descent_method_clear_cython_verified(A, f, 10)

### 4) numpy + Cython

In [ ]:
%%cython -a
import warnings
import numpy as np
from typing import Union


def steepest_descent_method_numpy_cython(A_arg: np.matrix, f_arg: np.array, K_max: int) -> np.array:
    A, f = np.copy(A_arg), np.copy(f_arg)
    if not np.all(np.linalg.eigvals(A) > 0):
        warnings.warn("Matrix is not positive definite")
        return
    elif not np.allclose(A, A.T):
        warnings.warn("Matrix is not symmetric")
        return
    elif K_max < 0:
        warnings.warn("The number of iterations cannot be negative")
        return
    x = np.zeros(f.shape, dtype=np.dtype(np.float64))
    for k in range(K_max):
        r = np.squeeze(np.asarray(f - np.matmul(A, x)))
        alpha = (np.dot(r, r)/np.dot(np.matmul(A, r), r)).item(0)
        x = x + alpha * r
    return x

In [ ]:
steepest_descent_method_numpy_cython_verified = timeit_deco()(steepest_descent_method_numpy_cython)
steepest_descent_method_numpy_cython_verified(A, f, 10)

### 5) Итоги

In [ ]:
pd.read_json("steepest_descent_verified_data.json").sort_values(by="time (sec)").style.set_properties(**{'font-size': '16pt'})